### Importig dependencies

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import sqlite3
import pickle

import time
import re
import nltk
from nltk.corpus import stopwords

### Reading the data

In [41]:
conn = sqlite3.connect(r'C:\Users\DELL\Desktop\Applied AI\SVM\Self project\foodreview-svm\database.sqlite')

filtered_data = pd.read_sql_query("""SELECT * FROM Reviews where Score!=3""",conn)
conn.close()

In [42]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [43]:
filtered_data.shape

(525814, 10)

#### Classifying data into positive & negative reviews

In [44]:
filtered_data['Score'] = filtered_data['Score'].map(lambda x: 1 if x>3 else 0)

In [45]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


### Removing the duplicate features

In [46]:
filtered_data["Time"].nunique()

3157

In [47]:
filtered_data[(filtered_data["UserId"] == "A74SHV5ZD3RLT") & (filtered_data['Summary'] == 'The price is right')] # Example of duplicate data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
8166,8935,B0007A0AP8,A74SHV5ZD3RLT,"R. Ellis ""Bobby""",15,15,1,1303862400,The price is right,We have a little Maltese that we spoil to no e...
162162,175817,B0014DUUFC,A74SHV5ZD3RLT,"R. Ellis ""Bobby""",15,15,1,1303862400,The price is right,We have a little Maltese that we spoil to no e...
494170,534267,B0007A0AOY,A74SHV5ZD3RLT,"R. Ellis ""Bobby""",15,15,1,1303862400,The price is right,We have a little Maltese that we spoil to no e...
504729,545770,B001E5E1C8,A74SHV5ZD3RLT,"R. Ellis ""Bobby""",15,15,1,1303862400,The price is right,We have a little Maltese that we spoil to no e...


In [48]:
final = filtered_data.drop_duplicates(subset = ["UserId", "Text"])
final.shape

(363859, 10)

In [49]:
# Helpfulness Denominator should greater than or equal to Helpfulness Numerator
final = final[final["HelpfulnessDenominator"] >= final["HelpfulnessNumerator"]]
final.shape

(363857, 10)

In [50]:
# Checking number of possitive & negative reviews
final["Score"].value_counts()

1    306779
0     57078
Name: Score, dtype: int64

### Cleaning text reviews

In [51]:
i = 0
for sen in final ['Text'].values:
    if(len(re.findall('<.*?>', sen))):
        print(sen,"\n\n")
        i += 1
    if i == 5:
        break

I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service! 


Twizzlers, Strawberry my childhood favorite candy, made in Lancaster Pennsylvania by Y & S Candies, Inc. one of the oldest confectionery Firms in the United States, now a Subsidiary of the Hershey Company, the Company

In [52]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'yourselves', "won't", 'wasn', 'under', 'on', 'my', 'he', 'our', 'too', 'mustn', 'i', 'herself', 'shan', 'be', 'she', "mightn't", 'them', 've', 'do', 'm', "shan't", 'being', 'me', 'because', 'was', 'myself', 'ourselves', 'isn', 'the', 'its', 'don', "couldn't", 'an', 'this', 'ours', 'you', 'both', 'there', 'own', 'all', 'very', "she's", 'other', 'down', 'does', 'needn', "should've", 'weren', "isn't", 'why', 're', 'which', 'now', 'over', 'between', 'more', 'if', 'further', "needn't", "you're", 'should', "aren't", 't', 'into', 'where', 'about', "that'll", 'y', 'not', "wasn't", 'hers', 'any', 'mightn', "mustn't", 'few', 'before', 'will', 'had', 'by', 'themselves', 'whom', 'wouldn', 'here', 'such', 'are', "weren't", 'himself', 'with', 'to', 'once', "don't", 'is', 'aren', 'nor', "doesn't", 'their', 'then', 'from', 'shouldn', 'for', 'a', 'some', 'yourself', "you've", 'when', 'hadn', 'same', 'doing', 'so', 'who', 'have', 'again', 'itself', "haven't", 'or', 'through', 's', 'did', 'll', 'while'

In [54]:
# sample stemming
snow = nltk.stem.SnowballStemmer('english')
print(snow.stem('gracefully'))

grace
